## 3d plate capacitor

In [1]:
from netgen.occ import *
from ngsolve.webgui import Draw
from ngsolve import Mesh

In [2]:
# gen mesh
el1 = Box(Pnt(-0.5, -0.5, -0.6), Pnt(0.5, 0.5, -0.5)).bc("bc_el1").mat("egal")
el2 = Box(Pnt(-0.5, -0.5, 0.5), Pnt(0.5, 0.5, 0.6)).bc("bc_el2").mat("egal")
air = Box(Pnt(-1, -1, -1), Pnt(1, 1, 1)) - el1 - el2

el1.mat("unknwn")
el2.mat("unknwn")
air.mat("air")
geo = OCCGeometry(Glue([air, el1, el2]))
mesh = Mesh(geo.GenerateMesh(maxh=1))

In [3]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [4]:
print(mesh.GetBoundaries())

('default', 'default', 'default', 'default', 'default', 'default', 'bc_el1', 'bc_el1', 'bc_el1', 'bc_el1', 'bc_el1', 'bc_el1', 'bc_el2', 'bc_el2', 'bc_el2', 'bc_el2', 'bc_el2', 'bc_el2')


In [5]:
print(mesh.GetMaterials())

('air', 'unknwn', 'unknwn')


In [6]:
from collections import defaultdict
from ngsolve import CoefficientFunction

In [7]:
diel_perm = defaultdict(lambda: -1)  # -1 ist default
# all diel_perm["unknwn"] = -1
diel_perm["air"] = 1.0
diel_perm_cf = CoefficientFunction([diel_perm[mat] for mat in mesh.GetMaterials()])

voltage = 4
potential = defaultdict(lambda: 0)
potential["bc_el1"] = voltage / 2
potential["bc_el2"] = -voltage / 2
potential_cf = CoefficientFunction([potential[bnd] for bnd in mesh.GetBoundaries()])
potential_str = "bc_el1|bc_el2"

In [8]:
Draw(potential_cf, mesh, "Potential")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [11]:
dirichlet_str = "air|bc_el1|bc_el2"

In [12]:
from ngsolve import H1, BilinearForm, grad, dx, GridFunction, Grad

In [13]:
fes = H1(mesh, order=3, dirichlet=dirichlet_str)
ut, vt = fes.TnT()
a = BilinearForm(fes)

a += diel_perm_cf * grad(ut) * grad(vt) * dx
a.Assemble()
u = GridFunction(fes, name="Potential")
u.Set(potential_cf, definedon=mesh.Boundaries(potential_str))
f = u.vec.CreateVector()
f.data = a.mat * u.vec
u.vec.data -= a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * f

In [14]:
Draw(u)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [ ]:
# E = -Grad(u)
# Draw(E, mesh, "E")